In [1]:
# 這是您應該在 Google Colab 或有 GPU 的環境中運行的代碼

# =================================================================
# 步驟 1: 環境設置與安裝
# =================================================================
print("--- 1. 環境設置與安裝 ---")
# 安裝必要的函式庫
# 由於要處理韓文，建議使用專門的韓文模型
!pip install transformers pandas datasets accelerate -U
!pip install sentencepiece # 對於某些韓文模型可能需要

import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import torch

# 檢查 CUDA (GPU) 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用設備: {device}")




--- 1. 環境設置與安裝 ---
使用設備: cuda


In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset
# 假定其他必要的匯入 (transformers, etc.) 都在您的完整程式碼中

def clean_and_extract_year(date_str):
    # (日期清理函數保持不變)
    if pd.isna(date_str) or date_str == '':
        return None
    try:
        # 嘗試轉換完整的 'YYYY/MM/DD' 格式
        return pd.to_datetime(date_str, format='%Y/%m/%d').year
    except:
        pass
    if isinstance(date_str, (int, float)):
        date_str = str(int(date_str))

    if isinstance(date_str, str) and date_str.isdigit() and len(date_str) == 4:
        # 這是純年份，直接返回整數年份
        return int(date_str)

    return None

print("--- 2. 數據準備與篩選 (HOT 2010-2019) - 最終字元修正 ---")
try:
    df = pd.read_csv("final_SM.csv")

    # 🚨 修正步驟 1：重新創建 'full_lyrics' 欄位
    original_lyrics_col = 'lyrics'

# 直接使用原始歌詞
    df['full_lyrics'] = df[original_lyrics_col].fillna("").astype(str)



    # 修正步驟 2：執行日期清洗和年份提取
    df['release_year'] = df['release_date'].apply(clean_and_extract_year)
    df = df.dropna(subset=['release_year'])
    df['release_year'] = df['release_year'].astype(int)

    # 🚨 修正步驟 3：精確匹配您確認的字元 Girls’ Generation
    # 我們將使用兩種常見的單引號格式來確保匹配成功

    # 設置目標團體名稱：使用您確認的右單引號
    target_group_accurate = 'H.O.T'

    # 使用包含判斷 (模糊匹配) 來應對各種名稱變體 (如 Girls' Generation (TTS) 等)
    is_artist = df['recording_artist_credit'].astype(str).str.contains(target_group_accurate, case=False, na=False)

    # 額外判斷：確保也能匹配標準撇號 '
    target_group_fallback = "H.O.T"
    is_artist_fallback = df['recording_artist_credit'].astype(str).str.contains(target_group_fallback, case=False, na=False)

    # 合併兩種匹配結果
    final_artist_match = is_artist | is_artist_fallback


    target_df = df[
        final_artist_match & # 使用優化的藝術家匹配邏輯
        (df['release_year'] >= 2000) &
        (df['release_year'] <= 2019)
    ].copy()

    # 檢查結果
    target_df = target_df[['full_lyrics']].dropna()

    if target_df.empty:
        print(f"警告: 篩選條件太嚴格或數據不符。在 '{target_group_accurate}' ({2010}-{2019}) 年間未找到歌詞數據。")
        print("請檢查：1. 數據中是否還有其他名稱拼寫？ 2. 您的 'release_year' 欄位是否正確提取年份？")
    else:
        print(f"✅ 篩選出 {len(target_df)} 筆目標歌詞數據。")
        # 繼續 Dataset 轉換...
        # lyrics_dataset = Dataset.from_pandas(target_df.reset_index(drop=True))

except FileNotFoundError:
    print("致命錯誤: 找不到 'final_SM.csv' 檔案。")
    exit()
except KeyError as e:
    print(f"致命錯誤: 欄位名稱錯誤。請檢查您的 CSV 檔案中是否缺少必要欄位。錯誤: {e}")
    exit()
except Exception as e:
    print(f"發生一般錯誤: {e}")
    exit()
lyrics_dataset = Dataset.from_pandas(target_df.reset_index(drop=True))

--- 2. 數據準備與篩選 (HOT 2010-2019) - 最終字元修正 ---
✅ 篩選出 20 筆目標歌詞數據。


In [3]:
# 選擇最穩定、官方支持的韓文生成模型
model_id = "skt/kogpt2-base-v2"  # <-- 最終建議使用這個標準模型

try:
    print(f"嘗試載入標準 KoGPT2 模型: {model_id}")
    # 使用 Auto 類別，因為官方模型配置完整
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

    # 設置 Tokenizer 的 Pad Token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("✅ Tokenizer 和模型已成功載入。")

except Exception as e:
    print(f"❌ 載入 {model_id} 時發生錯誤: {e}")
    print("請確認您的 transformers 庫版本是否足夠新。")
    exit()

# ----------------------------------------------------------------

嘗試載入標準 KoGPT2 模型: skt/kogpt2-base-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Tokenizer 和模型已成功載入。


In [4]:
# -------------------- 步驟 3：模型載入與 Tokenizer 修正 --------------------

# 確保所有必要的函式庫已匯入（假設您已在程式頂部匯入）
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import torch

model_id = "skt/kogpt2-base-v2"  # 穩定且官方支持的 KoGPT2 模型
print(f"\n--- 3. 模型載入、分詞與切分 ---")
print(f"嘗試載入標準 KoGPT2 模型: {model_id}")

# 載入模型和 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

# 🚨 Tokenizer 修正：明確設定 Padding 參數，解決部分 CUDA 問題
tokenizer.padding_side = "right"
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    # 🚨 Padding 修正：強制填充到最大長度 128
    return tokenizer(
        examples["full_lyrics"],
        truncation=True,
        max_length=128,
        padding='max_length'
    )

print("正在進行分詞和數據切分...")

# 進行 Tokenizer
tokenized_datasets = lyrics_dataset.map(tokenize_function, batched=True, remove_columns=["full_lyrics"])

# 映射標籤 (Labels)
lm_datasets = tokenized_datasets.map(
    lambda x: {"labels": x["input_ids"]},
    batched=True
)

# 劃分訓練集和測試集
if len(lm_datasets) > 1:
    train_test_split = lm_datasets.train_test_split(test_size=0.1)
    train_dataset = train_test_split['train']
    eval_dataset = train_test_split['test']
    print(f"✅ 數據集已成功切分為訓練集 ({len(train_dataset)} 筆) 和驗證集 ({len(eval_dataset)} 筆)。")
else:
    # 避免 NameError，即使數據不足也將其設置為 None
    train_dataset = None
    eval_dataset = None
    print("❌ 數據量不足，訓練集和驗證集未定義。")


--- 3. 模型載入、分詞與切分 ---
嘗試載入標準 KoGPT2 模型: skt/kogpt2-base-v2


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


正在進行分詞和數據切分...


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

✅ 數據集已成功切分為訓練集 (18 筆) 和驗證集 (2 筆)。


In [5]:
print("\n--- 3.5 最終數據類型強制轉換 ---")

if train_dataset is not None:
    # 🚨 關鍵修正：將所有數值欄位強制轉換為 PyTorch Long 類型，解決底層 CUDA 錯誤
    train_dataset.set_format(
        type='torch',
        columns=['input_ids', 'attention_mask', 'labels'],
        output_all_columns=True,
        dtype=torch.long
    )
    print("✅ 訓練集數據類型已強制轉換為 torch.long。")

if eval_dataset is not None:
    eval_dataset.set_format(
        type='torch',
        columns=['input_ids', 'attention_mask', 'labels'],
        output_all_columns=True,
        dtype=torch.long
    )
    print("✅ 驗證集數據類型已強制轉換為 torch.long。")


--- 3.5 最終數據類型強制轉換 ---
✅ 訓練集數據類型已強制轉換為 torch.long。
✅ 驗證集數據類型已強制轉換為 torch.long。


In [6]:
if train_dataset is not None:
    print("\n--- 4. 模型微調 (Fine-tuning) 啟動 ---")

    # 1. 定義數據打包器 Data Collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False # Causal Language Modeling
    )

    training_args = TrainingArguments(
        output_dir="./snsd_lyrics_model",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        learning_rate=5e-5,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,

        # 🚨 關鍵修正：禁用混合精度和 wandb
        fp16=False,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
    )

    print("🚨 正在啟動模型訓練 (trainer.train())，請等待... 🚨")
    # 啟動訓練！
    trainer.train()

    # 訓練完成後儲存模型
    trainer.save_model("./snsd_lyrics_model_finetuned")
    print("\n✅ 模型訓練完成並已儲存至 './snsd_lyrics_model_finetuned' 資料夾。")
else:
    print("\n❌ 訓練無法開始：訓練數據集為空或不足。請回到步驟 2 修正數據篩選條件。")


--- 4. 模型微調 (Fine-tuning) 啟動 ---
🚨 正在啟動模型訓練 (trainer.train())，請等待... 🚨


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss



✅ 模型訓練完成並已儲存至 './snsd_lyrics_model_finetuned' 資料夾。


In [13]:
# =================================================================
# 步驟 5: 載入微調後的模型並生成歌詞
# =================================================================

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. 設定路徑與設備
fine_tuned_model_path = "./snsd_lyrics_model_finetuned"  # 您剛剛儲存模型的路徑
base_model_id = "skt/kogpt2-base-v2"                     # 原始模型 ID (用於載入 Tokenizer)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("正在載入微調後的模型...")
try:
    # 載入您訓練好的模型權重
    model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path).to(device)
    # 載入 Tokenizer (通常使用原始基礎模型的 Tokenizer 即可)
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    print("✅ 模型載入成功！")
except Exception as e:
    print(f"❌ 載入失敗: {e}")
    print("請確認 './snsd_lyrics_model_finetuned' 資料夾是否存在。")
    exit()

# 2. 定義生成函式
def generate_lyrics(prompt_text, max_length=150, temperature=0.7, top_k=50):
    model.eval() # 設定為評估模式

    # 編碼提示詞
    input_ids = tokenizer.encode(prompt_text, return_tensors='pt').to(device)

    # 生成
    with torch.no_grad():
        output_sequences = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            temperature=temperature, # 創意度：越高越隨機 (0.7-1.0 適合歌詞)
            top_k=top_k,             # 採樣策略
            top_p=0.9,
            repetition_penalty=1.2,  # 防止重複懲罰
            do_sample=True,          # 啟用隨機採樣
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # 解碼生成的文字
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return generated_text

# =================================================================
# 3. 開始生成！試試看輸入一些少女時代風格的關鍵詞
# =================================================================

# 範例提示詞 (您可以自由修改)
prompts = [
    "Baby, I love you",
    "I am so hot",
    '이제 모든 것을'
     # 抒情風格
]

print(f"\n--- 🎶HOT (2010-2019) 風格歌詞生成結果 🎶 ---\n")

for p in prompts:
    print(f"🔹 提示詞: {p}")
    lyrics = generate_lyrics(p, max_length=100)
    print(f"✨ 生成歌詞:\n{lyrics}\n")
    print("-" * 50)

正在載入微調後的模型...
✅ 模型載入成功！

--- 🎶HOT (2010-2019) 風格歌詞生成結果 🎶 ---

🔹 提示詞: Baby, I love you
✨ 生成歌詞:
Baby, I love you gottain
My movie, a fucking out!
I can't know the needs to want into it's biggest for this handline.
Here's so down only part of devices in Little House
That's tell stop! It's everydays, and

--------------------------------------------------
🔹 提示詞: I am so hot
✨ 生成歌詞:
I am so hot to me can't buy it know a little party, what did no, even's getsome into the factory
Tip this satisfaction with your experience, twice from the stage
And anything form of handlife, Him-shinton and John Gill

--------------------------------------------------
🔹 提示詞: 이제 모든 것을
✨ 生成歌詞:
이제 모든 것을 다 잘 할 수 있기를 바란다.
그런데 당신들의 앞날은 결코 밝지 않다.
이제 저들도 모두들 이 길을 가는 것 자체가 두려운 일이다.
하지만 우리는 지금 모두가 내 앞에 놓여 있는 이러한 현실에 맞서고 있다.
우리가 함께 힘을 모아야 한다.
우리는 서로를 이해하고 용서하며 그 사랑을 다시 한 번 확인하는 것이 우리의 의무이다.
우리에게 주어진 마지막 남은 선택
나의 두 번째 임무가 바로 나의 존재 이유의 확인에 있습니다.
더 이상 나 자신을 믿지 말아주

--------------------------------------------------


In [8]:
print("--- 訓練數據抽樣檢查 ---")
sample_text = tokenizer.decode(train_dataset[0]['input_ids'], skip_special_tokens=True)
print(sample_text)

--- 訓練數據抽樣檢查 ---
이제 모든 것을 다 모두가
기다려주지 않는다는 것에 이런 현실 속에(속에)
새롭게 바꿔 가야하는 나의 의지와
결코 포기하지 않는uh! 우리들의 굳은 결의(결의)
이것으로 또 모두가 지켜내야 할(내가 할)
우리들의 미래 life concept tic toc to the OP.T
내게 기다린 내게 이제와 살아갈 수 있게
한없이 핏빛에 물들 것은 이제 없게
우리가 돌아갈 수 있는 길은 없다
오직 앞만을 향해
꺼져 가는 모든 것을 이제 내가 빠짐없이 비춰주리라
언제나 기다려왔어 너 없는


In [9]:
# =================================================================
# 無條件生成 (Unconditional Generation)
# =================================================================

import torch

# 1. 準備「開始信號」
# 大多數模型都有一個特殊的 "BOS" (Beginning of Sentence) token
# 如果模型沒有設定 BOS，我們就用 EOS (End of Sentence) 代替，效果通常一樣
start_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id

# 建立只有一個開始符號的輸入
input_ids = torch.tensor([[start_token]], device=device)

# 2. 生成設定
print("🎶 正在隨機創作HOT風格歌詞 (無提示詞)... 🎶\n")

model.eval()
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=200,          # 設定長一點，讓它寫完一段
        temperature=0.8,         # 稍微高一點的創意度
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,  # 防止鬼打牆
        do_sample=True,          # 隨機採樣
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

# 3. 解碼結果
generated_lyrics = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_lyrics)

🎶 正在隨機創作HOT風格歌詞 (無提示詞)... 🎶



#blacknow .
#singapore #movie #instagood #instafashion 오늘은 왜이렇게 맛난거야~
아침부터 이쁘다 생각하셔서 얼른 먹기로 했어요
그래도 아까워서 못먹겠어서 더 안먹었더니 이렇게 맛있는게 또 어디있어요!
얼마나 먹고싶었는데 너무 배가 불렀는지
새벽까지 고생이 많으시던 할머니와
저녁까지 밖에서 꼬박꼬박 꼬박 챙겨주시는 오빠. 정말 감사합니당 ᄒᄒ
오늘은 꼭꼭 먹어야할 것 같은데용! 우리 가족들이랑 오실 수 있는 메뉴가 있어서
한번쯤 한번에 먹어볼 만한 음식이에요 
메인으로는 두부구이 / 콩나물무침, 버섯샐러드, 그리고 고추장찌개, 닭가슴살볶음밥 / 샐러드바


In [10]:
# 檢查您的數據集內容
print("--- 🔍 數據集內容檢查 ---")

# 隨機查看 3 筆訓練數據的原始文本
import random
for i in range(3):
    try:
        # 我們將 input_ids 轉回文字，看看模型實際學到了什麼
        sample_id = random.randint(0, len(train_dataset) - 1)
        decoded_text = tokenizer.decode(train_dataset[sample_id]['input_ids'], skip_special_tokens=True)
        print(f"\n📄 樣本 {i+1}:")
        print(decoded_text[:200] + "...") # 只印前200字
    except Exception as e:
        print(f"讀取失敗: {e}")

--- 🔍 數據集內容檢查 ---

📄 樣本 1:
미안해 네게 상처 줬던 것 착하기만 하던
너의 하얀 두 볼 위에서 눈물 흘리게 한 것
용서해 용서해야만 해요 그대 떠나가는 게
나는 너무 두려웠기에 너무 사랑했기에
한땐 그대가 내 곁에 없는 게
너무나 힘이 들고 또 외로워서
비가 내리던 날 난 비를 맞으며
한없이 울었었죠
그대를 사랑해요 보고 싶어요
그댈 만나 행복했던 것만큼
내가 약속할게요 이제 다시는
...

📄 樣本 2:
이제 모든 것을 다 모두가
기다려주지 않는다는 것에 이런 현실 속에(속에)
새롭게 바꿔 가야하는 나의 의지와
결코 포기하지 않는uh! 우리들의 굳은 결의(결의)
이것으로 또 모두가 지켜내야 할(내가 할)
우리들의 미래 life concept tic toc to the OP.T
내게 기다린 내게 이제와 살아갈 수 있게
한없이 핏빛에 물들 것은 이제 없게
우리...

📄 樣本 3:
Pump up the track some for your partner

Lets see how many rappers can go the length
Back in your system with extra strength
We eat emcees, outdo and surpass em
But what's gonna make ya different than...
